<a href="https://colab.research.google.com/github/SashaNasonova/burnSeverity/blob/main/BurnSeverity_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Burn Severity Mapping Notebook
This notebook is intended to be used for small scale, manual burn severity mapping of individual fires. For large scale semi-automated mapping please refer to the main python scripts (https://github.com/SashaNasonova/burnSeverity).

This methodology is based on the Burned Area Reflectance Classification (BARC) product developed by the USGS that aims to estimate burn severity through a spectral comparison of pre- and post-fire medium resolution (20 - 30m) satellite imagery.

Healthy vegetation reflects strongly in the near-infrared (NIR) portion of the electromagnetic spectrum whereas rock and bare soil reflects strongly in the mid to shortwave infrared (SWIR) portion. In other words, healthy vegetation reflects strongly in NIR and reflects weakly in SWIR **(↑NIR,↓SWIR)**, whereas soil, bare rock and burned woody vegetation reflect strongly in SWIR and weakly in NIR **(↑SWIR,↓NIR)**. This inverse relationship can be leveraged to provide an estimate of burn severity where both pre- and post-fire imagery is available.

The Normalized Burn Ratio (NBR) is a spectral index that captures the relationship between NIR and SWIR bands. The difference between pre- and post-fire NBR (dNBR) can then be used to quantify wildfire burn severity (**↑dNBR ∝ ↑Severity**) using the following equations.

(1) NBR = (NIR - SWIR) / (NIR + SWIR) \\
(2) dNBR = NBRpre - NBRpost

Once dNBR has been calculated, it can be transformed into a burn severity classification product using a variety of methods ranging from simple thresholding to more complex supervised classifications informed by ground observations. This process is based on the USGS BARC256 methodology which scales the data to an 8-bit representation and utilizes static thresholds (76,110,187) to create a burn severity classification from the dNBR raster.


In [ ]:
# Clone github repository to be able to access vector data
!git clone https://github.com/SashaNasonova/burnSeverity.git

In [ ]:
#Install geemap through pip
%pip install -U geemap
%pip install pycrs

In [ ]:
#Import dependencies
import geemap
import ee
import os
import geopandas
from osgeo import gdal
from google.colab import files

In [ ]:
#Authenticate gee
ee.Authenticate()

In [ ]:
#Intialize with a google cloud project
project = 'helical-apricot-328020'
ee.Initialize(project=project)

In [ ]:
## Define functions
# Helper function to get files, non-recursive
def getfiles(d,ext):
  paths = []
  for file in os.listdir(d):
      if file.endswith(ext):
          paths.append(os.path.join(d, file))
  return(paths)

# Helper function to get image acquisition date and format into ("yyyy-mm-dd")
def getDate(im):
  return(ee.Image(im).date().format("YYYY-MM-dd"))

def getSceneIds(im):
  return(ee.Image(im).get('PRODUCT_ID'))

def mosaicByDate(indate):
  d = ee.Date(indate)
  #print(d)
  im = col.filterBounds(poly).filterDate(d, d.advance(1, "day")).mosaic()
  #print(im)
  return(im.set("system:time_start", d.millis(), "system:index", d.format("YYYY-MM-dd")))

def runDateMosaic(col_list):
  #get a list of unique dates within the list
  date_list = col_list.map(getDate).getInfo()
  udates = list(set(date_list))
  udates.sort()
  udates_ee = ee.List(udates)

  #mosaic images by unique date
  mosaic_imlist = udates_ee.map(mosaicByDate)
  return(ee.ImageCollection(mosaic_imlist))

def NBR_S2(image):
  nbr = image.expression(
      '(NIR - SWIR) / (NIR + SWIR)', {
          'NIR': image.select('B8'),
          'SWIR': image.select('B12')}).rename('nbr')
  return(nbr)

def NBR_Landsat(image,dattype):
  if (dattype == 'L5')|(dattype == 'L7'):
      nbr = image.expression(
          '(NIR - SWIR) / (NIR + SWIR)', {
              'NIR': image.select('SR_B4'),
              'SWIR': image.select('SR_B7')}).rename('nbr')
  elif (dattype == 'L8')|(dattype == 'L9'):
      nbr = image.expression(
          '(NIR - SWIR) / (NIR + SWIR)', {
              'NIR': image.select('SR_B5'),
              'SWIR': image.select('SR_B7')}).rename('nbr')
  else:
      print('Incorrect Landsat sensor specified')
  return(nbr)

def grid_footprint(footprint,nx,ny):
  from shapely.geometry import Polygon, LineString, MultiPolygon
  from shapely.ops import split

  #polygon = footprint
  polygon = Polygon(footprint['coordinates'][0])
  #polygon = Polygon(footprint)

  minx, miny, maxx, maxy = polygon.bounds
  dx = (maxx - minx) / nx  # width of a small part
  dy = (maxy - miny) / ny  # height of a small part

  horizontal_splitters = [LineString([(minx, miny + i*dy), (maxx, miny + i*dy)]) for i in range(ny)]
  vertical_splitters = [LineString([(minx + i*dx, miny), (minx + i*dx, maxy)]) for i in range(nx)]
  splitters = horizontal_splitters + vertical_splitters

  result = polygon
  for splitter in splitters:
      result = MultiPolygon(split(result, splitter))

  coord_list = [list(part.exterior.coords) for part in result.geoms]

  poly_list = []
  for cc in coord_list:
      p = ee.Geometry.Polygon(cc)
      poly_list.append(p)
  return(poly_list)

def apply_scale_factors_ls(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(opticalBands, None, True).addBands(thermalBands, None, True)

def apply_scale_factors_s2(image):
  opticalBands = image.select('B.*').multiply(0.0001)
  return image.addBands(opticalBands, None, True)


In [ ]:
# Open fires shapefile
fires_shp = 'burnSeverity/test/vectors/K52318_Aug28.shp'
fires = geemap.shp_to_ee(fires_shp)

# Visualize in table format
fires_df = geopandas.read_file(fires_shp)
fires_df

In [ ]:
# Visualize if needed
Map = geemap.Map()
Map.addLayer(fires,{},'Fire Polys')
Map.centerObject(fires)
Map

In [ ]:
# Select data type
dattype = 'S2'

if dattype == 'S2':
    col = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    cld_field =  'CLOUDY_PIXEL_PERCENTAGE'
    print('Selected S2 SR')
elif dattype == 'L9':
    col = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    cld_field = 'CLOUD_COVER'
    print('Selected L9 SR')
elif dattype == 'L8':
    col = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
    cld_field = 'CLOUD_COVER'
    print('Selected L8 SR')
else:
    print('Wrong Data Type Selected')


In [ ]:
# Define visualization parameters
vis_type = 'nir'

# Dictionary with band combinations
vis_dict = {'L8':{'nir':['SR_B5', 'SR_B4', 'SR_B3'],
                  'tc':['SR_B4','SR_B3','SR_B2'],
                  'swir':['SR_B7','SR_B6','SR_B4']},
            'L9':{'nir':['SR_B5', 'SR_B4', 'SR_B3'],
                  'tc':['SR_B4','SR_B3','SR_B2'],
                  'swir':['SR_B7','SR_B6','SR_B4']},
            'S2':{'nir':['B8', 'B4', 'B3'],
                  'tc':['B4','B3','B2'],
                  'swir':['B12','B8','B4']}}

bands = vis_dict[dattype][vis_type] #or provide a list of 3 bands

vis = {
  'min': 0,
  'max': 0.4,
  'bands': bands,
};

In [ ]:
# Now select one fire (in the test data, there's only one fire perimeter)
firenumber = 'K52318'
fieldname = 'FIRE_NUMBE' #unique firenumber field, change if needed
poly = fires.filterMetadata(fieldname,'equals',firenumber)

**Part 3: Pre-image selection parameters**

In [ ]:
# Find pre-fire imagery
startdate = '2023-07-15'
enddate = '2023-07-30'
cld_thr = 100 #can lower the cloud cover threshold

before = col.filterDate(startdate,enddate).filterBounds(poly).filter(ee.Filter.lt(cld_field,cld_thr))

before_list = before.toList(before.size().getInfo())
pre_mosaic_col = runDateMosaic(before_list)

if dattype.startswith('L'):
  pre_mosaic_col = pre_mosaic_col.map(apply_scale_factors_ls)
else:
  pre_mosaic_col = pre_mosaic_col.map(apply_scale_factors_s2)

print('Found',pre_mosaic_col.size().getInfo(),'dates')

**Pre-image map display**

In [ ]:
# Add pre-fire images to pre-fire map
before_map = geemap.Map()

before_size = pre_mosaic_col.size().getInfo()
before_mosaic_list = pre_mosaic_col.toList(before_size)
before_size

for i in range(0,before_size):
  b = before_mosaic_list.get(i)
  date = b.getInfo()['properties']['system:index']
  before_map.addLayer(ee.Image(b), vis, date)
  print(date)

# Add fire polygon
style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
before_map.addLayer(poly.style(**style),{},'poly')
before_map.centerObject(poly)
before_map

At this point take note of the date of the pre-fire image you chose (yyyy-mm-dd). If there's no clear imagery available for the date you range you defined you can try a different date range or go back to select a different sensor (dattype). It is generally considered best practice to select pre- and post-fire imagery that was acquired at the time of year to reduce seasonal effects on the comparison, especially in areas with a high decidious component. ** add section details.

**Part 4: Post-image parameter selection**

In [ ]:
# Find post-fire images
startdate = '2023-09-15'
enddate = '2023-09-30' #not inclusive of the end date
cld_thr = 100

after = col.filterDate(startdate,enddate).filterBounds(poly).filter(ee.Filter.lt(cld_field,cld_thr))

after_list = after.toList(after.size().getInfo())
post_mosaic_col = runDateMosaic(after_list)

if dattype.startswith('L'):
  post_mosaic_col = post_mosaic_col.map(apply_scale_factors_ls)
else:
  post_mosaic_col = post_mosaic_col.map(apply_scale_factors_s2)

print('Found',post_mosaic_col.size().getInfo(),'scenes')

In [ ]:
# Add post-fire images to post-fire map
after_map = geemap.Map()

post_size = post_mosaic_col.size().getInfo()
post_mosaic_list = post_mosaic_col.toList(post_size)
before_size

for i in range(0,post_size):
  b = post_mosaic_list.get(i)
  date = b.getInfo()['properties']['system:index']
  after_map.addLayer(ee.Image(b), vis, date)
  print(date)

# Add fire polygon
style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
after_map.addLayer(poly.style(**style),{},'poly')
after_map.centerObject(poly)
after_map


Also take note of the post-fire image selected. If no good post-fire image is available consider changing the date range or the sensor for both pre- and post-fire image.

In [ ]:
# Calculate NBR, dNBR and generate BARC map.

pre_mosaic_date = '2023-07-23' #enter pre-fire image date here
post_mosaic_date = '2023-09-21' #enter post_fire image date here

# Select pre-image and post-image
pre_col = pre_mosaic_col.filter(ee.Filter.inList("system:index",ee.List([pre_mosaic_date])))
pre_img = ee.Image(pre_col.toList(1).get(0))

if pre_col.size().getInfo() != 1:
  raise ValueError("Didn't select 1 pre-fire image date. Check pre_mosaic_date!")

post_col = post_mosaic_col.filter(ee.Filter.inList("system:index", ee.List([post_mosaic_date])))
post_img = ee.Image(post_col.toList(1).get(0))

if post_col.size().getInfo() != 1:
  raise ValueError("Didn't select 1 post-fire image date. Check post_mosaic_date!")

# Calculate NBR
if dattype.startswith('S2'):
    pre_nbr = NBR_S2(pre_img)
    post_nbr = NBR_S2(post_img)
else:
    pre_nbr = NBR_Landsat(pre_img,dattype)
    post_nbr = NBR_Landsat(post_img,dattype)

# Calculate dNBR
dNBR = pre_nbr.subtract(post_nbr).rename('dNBR')

# Scale dNBR
dNBR_scaled = dNBR.expression('(dNBR * 1000 + 275)/5',{'dNBR': dNBR.select('dNBR')}).rename('dNBR_scaled')

# Classify
classes = dNBR_scaled.expression("(dNBR_scaled >= 187) ? 4 "
                                ": (dNBR_scaled >= 110) ? 3 "
                                ": (dNBR_scaled >= 76) ? 2 "
                                ": 1",{'dNBR_scaled': dNBR_scaled.select('dNBR_scaled')})


# Clip to fire perimeter
classes_clipped = classes.clip(poly)

print("BARC Complete")

In [ ]:
#Visualize pre- and post-fire imagery with BARC

barc_map = geemap.Map()
barc_map.addLayer(ee.Image(pre_img), vis, pre_mosaic_date)
barc_map.addLayer(ee.Image(post_img),vis, post_mosaic_date)

palette = ['000000','grey', 'yellow', 'orange','red']

barc_vis = {'min':0,'max':4,'palette':palette}
barc_map.addLayer(ee.Image(classes_clipped),barc_vis,'barc')

style = {'color': 'white', 'width': 2, 'lineType': 'solid', 'fillColor': '00000000'}
barc_map.addLayer(poly.style(**style),{},'poly')

barc_map.centerObject(poly)
barc_map

In [ ]:
# Export rasters
outfolder = firenumber
pre_date = pre_mosaic_date
post_date = post_mosaic_date

# Define spatial resolution
if dattype == 'S2':
  px = 20
else:
  px = 30

## Define tiling rules
poly_area = round(poly.geometry().area(1).divide(10000).getInfo(),1)
print(poly_area)
print('Fire Area:',poly_area,'hectares')

if poly_area < 10000:
    n = 2
elif poly_area > 10000 and poly_area < 100000:
    n = 3
elif poly_area > 100000 and poly_area < 400000:
    n = 4
else:
    n = 5

print('Number of tiles: ' + str(n*n))

#export pre and post rgbs, tile to avoid pixel limit issues.
footprint = poly.geometry().bounds().getInfo()
grids = grid_footprint(footprint,n,n)

for i in range(0,len(grids)):
  roi = grids[i]
  ## Export BARC
  barc_folder = os.path.join(outfolder,'barc')
  if not os.path.exists(barc_folder):
          os.makedirs(barc_folder)

  name = 'BARC_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype +'_' + str(i) + '_.tif'
  barc_filename = os.path.join(barc_folder,name)
  geemap.ee_export_image(classes_clipped.unmask(9).clip(roi), filename=barc_filename, scale=px, file_per_band=False,crs='EPSG:3005')
  ras = gdal.Open(barc_filename)
  dat = ras.GetRasterBand(1)
  dat.SetNoDataValue(9)
  ras = None
  dat = None

  ## Export 8-bit truecolor images
  #pre, truecolor
  pre_tc_8bit = os.path.join(outfolder,'pre_truecolor_8bit')
  if not os.path.exists(pre_tc_8bit):
          os.makedirs(pre_tc_8bit)
  filename = os.path.join(pre_tc_8bit, dattype + '_' + pre_date + '_truecolor_pre_8bit_' + str(i) + '.tif')
  pre_tc_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  post_tc_8bit = os.path.join(outfolder,'post_truecolor_8bit')
  if not os.path.exists(post_tc_8bit):
          os.makedirs(post_tc_8bit)
  filename = os.path.join(post_tc_8bit, dattype + '_' + post_date + '_truecolor_post_8bit_' + str(i) + '.tif')
  post_tc_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  ## Export swir too
  pre_sw_8bit = os.path.join(outfolder,'pre_swir_8bit')
  if not os.path.exists(pre_sw_8bit):
          os.makedirs(pre_sw_8bit)
  filename = os.path.join(pre_sw_8bit, dattype + '_' + pre_date + '_swir_pre_8bit_' + str(i) + '.tif')
  pre_sw_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B12', 'B8', 'B4'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(pre_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  post_sw_8bit = os.path.join(outfolder,'post_swir_8bit')
  if not os.path.exists(post_sw_8bit):
          os.makedirs(post_sw_8bit)
  filename = os.path.join(post_sw_8bit, dattype + '_' + post_date + '_swir_post_8bit_' + str(i) + '.tif')
  post_sw_8bit_path = filename
  if dattype.startswith('S2'):
      viz = {'bands': ['B12', 'B8', 'B4'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=10, file_per_band=False,crs='EPSG:3005')
  elif (dattype == 'L8') | (dattype == 'L9'):
      viz = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': 0, 'max':0.3,'gamma':1.5}
      geemap.ee_export_image(post_img.clip(roi).visualize(**viz), filename=filename, scale=30, file_per_band=False,crs='EPSG:3005')
  else:
      pass

  print(barc_folder)

#mosaic all
#BARC
barc_list = getfiles(barc_folder,'.tif')
outfilename = 'BARC_' + firenumber + '_' + pre_date + '_' + post_date + '_' + dattype + '.tif'
out = os.path.join(barc_folder,outfilename)
gdal.Warp(out,barc_list)
for file in barc_list: os.remove(file) #delete tiles
barc_filename = out #to return from the function
print('Barc mosaic complete')

#pre truecolour
pre_tc_list = getfiles(pre_tc_8bit,'.tif')
outfilename = dattype + '_' + pre_date + '_truecolor_pre_8bit' + '.tif'
out = os.path.join(pre_tc_8bit,outfilename)
gdal.Warp(out,pre_tc_list)
for file in pre_tc_list: os.remove(file) #delete tiles
pre_tc_8bit_path = out #to return from the function
print('Pre truecolor mosaic complete')

#post truecolor
post_tc_list = getfiles(post_tc_8bit,'.tif')
outfilename = dattype + '_' + post_date + '_truecolor_post_8bit' + '.tif'
out = os.path.join(post_tc_8bit,outfilename)
gdal.Warp(out,post_tc_list)
for file in post_tc_list: os.remove(file)
post_tc_8bit_path = out #to return from the function
print('Post truecolor mosaic complete')

#pre swir
pre_sw_list = getfiles(pre_sw_8bit,'.tif')
outfilename = dattype + '_' + pre_date + '_swir_pre_8bit' + '.tif'
out = os.path.join(pre_sw_8bit,outfilename)
gdal.Warp(out,pre_sw_list)
for file in pre_sw_list: os.remove(file) #delete tiles
pre_sw_8bit_path = out #to return from the function
print('Pre swir mosaic complete')

#post swir
post_sw_list = getfiles(post_sw_8bit,'.tif')
outfilename = dattype + '_' + post_date + '_swir_post_8bit' + '.tif'
out = os.path.join(post_sw_8bit,outfilename)
gdal.Warp(out,post_sw_list)
for file in post_sw_list: os.remove(file)
post_sw_8bit_path = out #to return from the function
print('Post swir mosaic complete')

#Zip to download
zip = firenumber + '.zip'
indata = r'/content/' + firenumber

!zip -r {zip} {indata}
files.download(zip)


In [ ]:
# Summary Section

# Create a summary table

# Create an inset map

# Create quicklooks

